In [ ]:
import tkinter as tk
import sounddevice as sd
from scipy.io.wavfile import write
import threading
import datetime
import numpy as np
import whisper
import os
import tkinter.messagebox as messagebox
from tkinter import filedialog


In [ ]:
fs = 44100  
recording = []
is_recording = False


In [ ]:
def start_recording():
    global recording, is_recording
    is_recording = True
    recording = []

    def callback(indata, frames, time, status):
        if is_recording:
            recording.append(indata.copy())

    stream = sd.InputStream(samplerate=fs, channels=1, callback=callback)
    stream.start()
    start_button.config(state="disabled")
    stop_button.config(state="normal")
    output_label.config(text="🎤 Recording...")

    def keep_stream_open():
        while is_recording:
            sd.sleep(100)
        stream.stop()
        stream.close()

    threading.Thread(target=keep_stream_open).start()


In [ ]:

def stop_recording():
    global is_recording
    is_recording = False
    start_button.config(state="normal")
    stop_button.config(state="disabled")
    output_label.config(text="🛑 Recording stopped...\n🔍 Transcribing...")

    audio = np.concatenate(recording, axis=0)

    temp_filename = "temp_recording.wav"
    write(temp_filename, fs, audio)

    model = whisper.load_model("base")
    result = model.transcribe(temp_filename)
    transcription = result["text"]

    output_label.config(text=f"📝 Transcription:\n{transcription}")

    save_file = messagebox.askyesno("Save Recording", "Do you want to save the audio recording as a .wav file?")
    
    if save_file:
        default_name = f"recording_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}.wav"
        file_path = filedialog.asksaveasfilename(defaultextension=".wav", filetypes=[("WAV files", "*.wav")], initialfile=default_name)

        if file_path:
            write(file_path, fs, audio)
            output_label.config(text=output_label.cget("text") + f"\n✅ Audio saved as: {file_path}")
        else:
            output_label.config(text=output_label.cget("text") + "\n⚠️ Save cancelled.")
    else:
        output_label.config(text=output_label.cget("text") + "\n⚠️ Audio was not saved.")


In [ ]:
root = tk.Tk()
root.title("🎙 Voice-to-Text Recorder")
root.geometry("400x300")

start_button = tk.Button(root, text="Start Recording", command=start_recording, bg="green", fg="white", font=('Arial', 12))
start_button.pack(pady=10)

stop_button = tk.Button(root, text="Stop Recording", command=stop_recording, bg="red", fg="white", font=('Arial', 12))
stop_button.pack(pady=10)
stop_button.config(state="disabled")

output_label = tk.Label(root, text="", wraplength=380, justify="left", font=('Arial', 10))
output_label.pack(pady=20)

root.mainloop()